In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# Import DoseMetrics modules
from dosemetrics.io import load_dose, load_mask
from dosemetrics.metrics.dvh import compute_dvh
from dosemetrics.metrics.scores import compute_conformity_index, compute_homogeneity_index
from dosemetrics.utils.plotting import plot_dvh, plot_multiple_dvhs
from dosemetrics.utils.compliance import check_constraint

print("✓ All imports successful!")

## 1. Data Organization

### Recommended Directory Structure

Organize your data like this:

```
my_data/
├── patient_001/
│   ├── dose.nii.gz          # 3D dose distribution
│   └── structures/           # Binary masks for each structure
│       ├── PTV.nii.gz
│       ├── Brain.nii.gz
│       ├── Brainstem.nii.gz
│       └── Optic_Chiasm.nii.gz
├── patient_002/
│   ├── dose.nii.gz
│   └── structures/
│       └── ...
└── ...
```

### File Format Requirements

**Dose files:**
- Format: NIfTI (`.nii.gz`), DICOM RT Dose (`.dcm`), or NRRD (`.nrrd`)
- Content: 3D volume with dose values in Gy or cGy
- Metadata: Must include voxel spacing and orientation

**Structure masks:**
- Format: NIfTI (`.nii.gz`), DICOM RT Struct (`.dcm`), or NRRD (`.seg.nrrd`)
- Content: Binary mask (0 = outside, 1 = inside) or labeled mask
- **Critical:** Must be spatially aligned with dose distribution

## 2. Specify Your Data Paths

Update these paths to point to your actual data:

In [ ]:
# UPDATE THESE PATHS TO YOUR DATA
patient_dir = Path("path/to/your/patient_data")  # Change this!
dose_file = patient_dir / "dose.nii.gz"          # Path to dose file
structures_dir = patient_dir / "structures"      # Directory with structure masks

# Optional: Specify structures of interest
structures_of_interest = [
    "PTV",           # Planning target volume
    "Brain",         # Brain
    "Brainstem",     # Brainstem
    "Optic_Chiasm",  # Optic chiasm
]

print(f"Patient directory: {patient_dir}")
print(f"Dose file: {dose_file}")
print(f"Structures directory: {structures_dir}")

## 3. Load Dose Distribution

Load the 3D dose distribution. DoseMetrics automatically handles:
- Format detection (NIfTI, DICOM, NRRD)
- Metadata extraction
- Unit conversion if needed

In [ ]:
# Load dose
try:
    dose = load_dose(dose_file)
    print(f"✓ Dose loaded successfully")
    print(f"  Shape: {dose.shape}")
    print(f"  Data type: {dose.dtype}")
    print(f"  Dose range: {dose.min():.2f} - {dose.max():.2f} Gy")
    print(f"  Mean dose: {dose.mean():.2f} Gy")
except FileNotFoundError:
    print(f"❌ Error: Dose file not found at {dose_file}")
    print("   Please update the path in the cell above.")
except Exception as e:
    print(f"❌ Error loading dose: {e}")

## 4. Load Structure Masks

Load all structure masks from the structures directory:

In [ ]:
# Load all structure masks
structures = {}

try:
    # Find all mask files
    mask_files = list(structures_dir.glob("*.nii.gz"))
    
    if not mask_files:
        print(f"⚠️  No .nii.gz files found in {structures_dir}")
    
    for mask_file in mask_files:
        structure_name = mask_file.stem.replace(".nii", "")
        
        # Skip if not in structures of interest (if specified)
        if structures_of_interest and structure_name not in structures_of_interest:
            continue
        
        mask = load_mask(mask_file)
        structures[structure_name] = mask
        
        # Print statistics
        volume_voxels = mask.sum()
        print(f"✓ Loaded {structure_name:20s} - {volume_voxels:7.0f} voxels")
    
    print(f"\n✓ Total structures loaded: {len(structures)}")
    
except FileNotFoundError:
    print(f"❌ Error: Structures directory not found at {structures_dir}")
except Exception as e:
    print(f"❌ Error loading structures: {e}")

## 5. Verify Spatial Alignment

**Critical check:** Dose and masks must have the same shape and coordinate system!

In [ ]:
# Check spatial alignment
print("Spatial Alignment Check:")
print(f"Dose shape: {dose.shape}")

all_aligned = True
for name, mask in structures.items():
    aligned = mask.shape == dose.shape
    status = "✓" if aligned else "❌"
    print(f"{status} {name:20s} shape: {mask.shape}")
    if not aligned:
        all_aligned = False

if all_aligned:
    print("\n✓ All structures are properly aligned with dose!")
else:
    print("\n❌ WARNING: Some structures are not aligned with dose!")
    print("   This will cause incorrect results. Please check your data.")

## 6. Compute DVH for All Structures

Calculate dose-volume histograms for all loaded structures:

In [ ]:
# Compute DVH for each structure
dvh_results = {}

for name, mask in structures.items():
    try:
        dvh = compute_dvh(
            dose=dose,
            mask=mask,
            organ_name=name,
            bins=1000,
            dose_unit="Gy"
        )
        dvh_results[name] = dvh
        print(f"✓ DVH computed for {name}")
    except Exception as e:
        print(f"❌ Error computing DVH for {name}: {e}")

print(f"\n✓ DVH computed for {len(dvh_results)} structures")

## 7. Visualize DVH Curves

Create an interactive plot showing all DVH curves:

In [ ]:
# Plot all DVHs together
if dvh_results:
    fig = plot_multiple_dvhs(
        dvh_results,
        title="Dose-Volume Histogram Analysis",
        xlabel="Dose (Gy)",
        ylabel="Volume (%)"
    )
    fig.show()
else:
    print("No DVH data to plot")

## 8. Compute Quality Metrics (for Target)

Calculate plan quality indices for the planning target volume:

In [ ]:
# Specify target structure and prescription dose
target_name = "PTV"  # Change this to match your target name
prescription_dose = 60.0  # Change this to your prescription dose in Gy

if target_name in structures:
    ptv_mask = structures[target_name]
    
    # Compute conformity index
    ci = compute_conformity_index(
        dose=dose,
        ptv_mask=ptv_mask,
        prescription_dose=prescription_dose
    )
    
    # Compute homogeneity index
    hi = compute_homogeneity_index(
        dose=dose,
        ptv_mask=ptv_mask
    )
    
    print(f"Quality Metrics for {target_name}:")
    print(f"  Conformity Index: {ci:.3f}")
    print(f"  Homogeneity Index: {hi:.3f}")
    print(f"\nInterpretation:")
    print(f"  CI closer to 1.0 = better conformity")
    print(f"  HI closer to 0.0 = more homogeneous dose")
else:
    print(f"❌ Target structure '{target_name}' not found")
    print(f"   Available structures: {list(structures.keys())}")

## 9. Check Dose Constraints

Evaluate if your plan meets clinical dose constraints:

In [ ]:
# Define dose constraints (UPDATE THESE FOR YOUR PROTOCOL)
constraints = [
    {"organ": "Brainstem", "type": "max", "limit": 54.0, "unit": "Gy"},
    {"organ": "Optic_Chiasm", "type": "max", "limit": 55.0, "unit": "Gy"},
    {"organ": "Brain", "type": "mean", "limit": 30.0, "unit": "Gy"},
    {"organ": "PTV", "type": "D95", "limit": 57.0, "unit": "Gy"},  # 95% of PTV receives >= 57 Gy
]

# Check each constraint
print("Dose Constraint Compliance:")
print("-" * 60)

for constraint in constraints:
    organ = constraint["organ"]
    
    if organ not in dvh_results:
        print(f"⚠️  {organ:20s} - Not analyzed")
        continue
    
    try:
        is_compliant = check_constraint(dvh_results[organ], constraint)
        status = "✓ PASS" if is_compliant else "❌ FAIL"
        
        constraint_str = f"{constraint['type']} < {constraint['limit']} {constraint['unit']}"
        print(f"{status}  {organ:20s} {constraint_str}")
    except Exception as e:
        print(f"❌ Error checking {organ}: {e}")

## 10. Extract Dose Statistics

Get detailed dose statistics for each structure:

In [ ]:
# Compute dose statistics
statistics = []

for name, mask in structures.items():
    # Extract doses within structure
    structure_doses = dose[mask > 0]
    
    if len(structure_doses) == 0:
        continue
    
    stats = {
        "Structure": name,
        "Volume (voxels)": mask.sum(),
        "Min Dose (Gy)": structure_doses.min(),
        "Max Dose (Gy)": structure_doses.max(),
        "Mean Dose (Gy)": structure_doses.mean(),
        "Median Dose (Gy)": np.median(structure_doses),
        "D95 (Gy)": np.percentile(structure_doses, 95),
        "D2 (Gy)": np.percentile(structure_doses, 98),
    }
    statistics.append(stats)

# Create DataFrame
stats_df = pd.DataFrame(statistics)
print("\nDose Statistics Summary:")
print(stats_df.to_string(index=False))

## 11. Export Results

Save your analysis results to files:

In [ ]:
# Create output directory
output_dir = patient_dir / "results"
output_dir.mkdir(exist_ok=True)

# Save dose statistics
stats_file = output_dir / "dose_statistics.csv"
stats_df.to_csv(stats_file, index=False)
print(f"✓ Dose statistics saved to: {stats_file}")

# Save DVH data
for name, dvh in dvh_results.items():
    dvh_file = output_dir / f"dvh_{name}.csv"
    dvh.to_csv(dvh_file, index=False)
    print(f"✓ DVH for {name} saved to: {dvh_file}")

# Save DVH plot
if dvh_results:
    fig = plot_multiple_dvhs(dvh_results, title="DVH Analysis")
    plot_file = output_dir / "dvh_plot.html"
    fig.write_html(plot_file)
    print(f"✓ Interactive DVH plot saved to: {plot_file}")

print(f"\n✓ All results saved to: {output_dir}")

## Summary

You've now learned how to:

✅ Organize your radiotherapy data
✅ Load dose distributions and structure masks
✅ Compute dose-volume histograms
✅ Calculate quality metrics
✅ Check constraint compliance
✅ Generate visualizations
✅ Export analysis results

## Next Steps

- **Compare plans**: Compare multiple treatment plans (TPS vs predicted)
- **Batch processing**: Analyze multiple patients efficiently
- **Custom constraints**: Define your own institutional constraints
- **Advanced metrics**: Explore gamma analysis and geometric comparisons

## Need Help?

- [Documentation](https://contouraid.github.io/dosemetrics/)
- [API Reference](https://contouraid.github.io/dosemetrics/api/)
- [Try Live Demo](https://huggingface.co/spaces/contouraid/dosemetrics)
- [GitHub Issues](https://github.com/contouraid/dosemetrics/issues)

Happy analyzing! 🎉